In [ ]:
import deepspeech
import numpy as np
import wave
import os

In [ ]:
MODEL_PATH = 'path/to/deepspeech-model.pbmm'
SCORER_PATH = 'path/to/deepspeech-model.scorer'

In [ ]:
# Load the DeepSpeech model
ds = deepspeech.Model(MODEL_PATH)
ds.enableExternalScorer(SCORER_PATH)

In [ ]:
# Function to read and process audio files for DeepSpeech
def read_audio(file_path):
    with wave.open(file_path, 'rb') as wf:
        # Ensure the audio file has the correct format (16kHz, 16-bit PCM)
        if wf.getframerate() != 16000:
            raise ValueError('Audio file must be 16kHz')
        if wf.getsampwidth() != 2:
            raise ValueError('Audio file must be 16-bit PCM')
        if wf.getnchannels() != 1:
            raise ValueError('Audio file must be mono')

        # Read and return the audio as numpy array
        audio = np.frombuffer(wf.readframes(wf.getnframes()), dtype=np.int16)
        return audio

In [ ]:
# Function to transcribe audio using DeepSpeech
def transcribe_deepspeech(audio_file):
    # Read the audio file
    audio = read_audio(audio_file)
    
    # Perform inference with DeepSpeech
    transcription = ds.stt(audio)
    return transcription

In [ ]:
# Example usage: loop through all audio files in a directory and transcribe them
def batch_transcribe(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".wav"):
            audio_file = os.path.join(directory, filename)
            transcription = transcribe_deepspeech(audio_file)
            print(f"Transcription for {filename}: {transcription}")